In [15]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Sequential,Model
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l2, l1
from keras import optimizers
import numpy as np
from PIL import Image
import json


In [2]:
model = VGG16(weights='imagenet', include_top=False,  input_shape=(256, 256, 3))


Instructions for updating:
Colocations handled automatically by placer.


In [3]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers:
    layer.trainable = False


In [4]:
x=model.output #?? quanti layer densi devo usare? e con quale numero di neuroni
x=Flatten(input_shape=model.output_shape[1:])(x)
x=Dense(256, activation='relu',  kernel_regularizer=l2(0.01))(x)
x=Dropout(0.5)(x)
preds=Dense(1, activation='sigmoid')(x) #dovrei usare softmax con due neuroni come output??

final_model=Model(inputs=model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
final_model.summary()

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255) #divide all pixel value for 225


#questo generator sembra creare immagini custom a partire dalla cartella specificata
train_generator=train_datagen.flow_from_directory('car-damage-dataset/data1a/training',
                                                 target_size=(256,256),
                                                 color_mode='rgb',
                                                 batch_size=20, #number of images for cycle
                                                 class_mode='binary',
                                                 shuffle=False)


'''A DirectoryIterator yielding tuples of (x, y) where x is a numpy array containing a batch of 
images with shape (batch_size (a.k.a. # of images), target_size, channels) 
and y is a numpy array of corresponding labels.

example of image 2x2 
[BATCH
    [   IMAGE1
        [    ROW1
            [r,g,b pixel (1,1)] , [r,g,b pixel(1,2)]
        ],
        [    ROW2
            [r,g,b pixel (1,2)] , [r,g,b pixel(2,2)]
        ]
    ],
    [   IMAGE2
        .....
    ]
]

batches[0] ->  x the numpy array 
batches[0][0] -> first image
batches[0][1] -> second image
batches[0][0][0] -> first image first row
batches[0][0][0][0] -> first image first row first pixel (rgb)
'''



Found 1840 images belonging to 2 classes.


'A DirectoryIterator yielding tuples of (x, y) where x is a numpy array containing a batch of \nimages with shape (batch_size (a.k.a. # of images), target_size, channels) \nand y is a numpy array of corresponding labels.\n\nexample of image 2x2 \n[BATCH\n    [   IMAGE1\n        [    ROW1\n            [r,g,b pixel (1,1)] , [r,g,b pixel(1,2)]\n        ],\n        [    ROW2\n            [r,g,b pixel (1,2)] , [r,g,b pixel(2,2)]\n        ]\n    ],\n    [   IMAGE2\n        .....\n    ]\n]\n\nbatches[0] ->  x the numpy array \nbatches[0][0] -> first image\nbatches[0][1] -> second image\nbatches[0][0][0] -> first image first row\nbatches[0][0][0][0] -> first image first row first pixel (rgb)\n'

In [7]:
validation_datagen = ImageDataGenerator(rescale=1./255) #divide all pixel value for 225


validation_generator = validation_datagen.flow_from_directory('car-damage-dataset/data1a/validation',
                                                           target_size=(256,256),
                                                           color_mode='rgb',
                                                           batch_size=20, #number of images for cycle
                                                           class_mode='binary',
                                                           shuffle=False)
    

Found 460 images belonging to 2 classes.


In [8]:
nb_train_samples = train_generator.n//train_generator.batch_size #number of image / batch size 
nb_validation_samples = validation_generator.n // validation_generator.batch_size

In [9]:
final_model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'])

In [11]:
fit_history = final_model.fit_generator(train_generator,
                          steps_per_epoch=nb_train_samples,
                          epochs=1,
                          validation_data=validation_generator,
                          validation_steps=nb_validation_samples,
                          verbose=1)

Epoch 1/1
92/92 [==============================] - 1950s 21s/step - loss: 5.8468 - acc: 0.5978 - val_loss: 5.6157 - val_acc: 0.8326


TEST

In [ ]:
i = 0

for batches in train_generator:
        i += 1
        if i > 1:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break
        #print(batches[0][0])
        #print(batches[1][0])
        img = Image.fromarray(batches[0][0], 'RGB')
        display(img)